In [2]:
api_key = 'RGAPI-426e6ce2-ad6c-48df-94f2-f21f59b9c156'

In [3]:
import requests
import pandas as pd
import numpy as np

api_url = 'https://na1.api.riotgames.com/tft/league/v1/entries/DIAMOND/IV?queue=RANKED_TFT&page=100'

api_url = api_url + '&api_key=' + api_key


/var/folders/0y/sv7y2fxn4nj33w_6ny49q83w0000gn/T/ipykernel_51770/1916509563.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [31]:
d4_response = requests.get(api_url)

In [116]:
# loop through every user in diamond 4 and add their names into a dataframe
page_num = 1
diamond_puuid = pd.DataFrame([])
while True:
    api_url = 'https://na1.api.riotgames.com/tft/league/v1/entries/DIAMOND/IV?queue=RANKED_TFT&page=' + str(page_num)
    api_url = api_url + '&api_key=' + api_key
    response = requests.get(api_url).json()
    if(response != []): 
        df = pd.DataFrame(response)
        df1 = df.loc[:, df.columns.intersection(['puuid'])]
        diamond_puuid = pd.concat([diamond_puuid, df1], axis=0)
    else:
        break

    page_num += 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [117]:
print(diamond_puuid.head(5))
# print(diamond_summoner_names.tail(5))
print(diamond_puuid.size)

                                               puuid
0  xAHUx3ABXJAB6CJvwk3I4f_fs59yufLEKIZ99o4N5dI0lo...
1  5CpXXSb9wmxCmvl-nJoc97HCqkegjSBtJmHUsJahpv4BXm...
2  OwBCsQUKzIhw11OcYuph1hmltOZAroDVlqFcmnFJwgzRNV...
3  c1Khe6uMdUMGs1JLRlCoMucPiMmlc9vdyAJq7xGvZQQydI...
4  XQnpgeSHnXvLlqDYClsGRdVrP7TQdj9Ir_aTPwi85hylLt...
9496


In [4]:
## get the puuids of the top 250 players in north america

import time

api_url = 'https://na1.api.riotgames.com/tft/league/v1/challenger?queue=RANKED_TFT'
api_url = api_url + '&api_key=' + api_key
challenger_players_response = requests.get(api_url).json()
challenger_summoner_ids = [challenger_players_response['entries'][i]['summonerId'] for i in range(0, len(challenger_players_response['entries']))]
challenger_puuids = []

count = 1
for id in challenger_summoner_ids:
    summoner_url = 'https://na1.api.riotgames.com/tft/summoner/v1/summoners/'+ id + '?api_key=' + api_key
    ## delete when have production key
    if count % 100 == 0:
        time.sleep(120)
        print("sleeping 120s")
    r = requests.get(summoner_url).json()
    challenger_puuids.append(r['puuid'])
    count += 1

sleeping 120s
sleeping 120s


In [209]:
print(len(challenger_puuids))

250


In [5]:
## get a list of all match ids within the last hour
import time
recent_match_ids = []
start_time = int(time.time()) - 36000

##
count = 1
for puuid in challenger_puuids:
    api_url_matches = 'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/' + puuid + '/ids?start=0&startTime=' + str(start_time) + '&count=20&api_key=' + api_key
    r = requests.get(api_url_matches).json()
    recent_match_ids = list(set(recent_match_ids + r))

    ## delete when have production key
    if count % 100 == 0:
        time.sleep(120)
        print("sleeping 120s")
    count += 1
    

sleeping 120s
sleeping 120s


In [6]:
print(recent_match_ids)

['NA1_4905653528', 'NA1_4905747867', 'NA1_4905731855', 'NA1_4905684888', 'NA1_4905572498', 'NA1_4905609997', 'NA1_4905761282', 'NA1_4905673038', 'NA1_4905701465', 'NA1_4905700198', 'NA1_4905713163', 'NA1_4905696478', 'NA1_4905566476', 'NA1_4905595661', 'NA1_4905639835', 'NA1_4905637682', 'NA1_4905694395', 'NA1_4905743665', 'NA1_4905574484', 'NA1_4905752197', 'NA1_4905643388', 'NA1_4905769436', 'NA1_4905550591', 'NA1_4905659111', 'NA1_4905707626', 'NA1_4905753920', 'NA1_4905723059', 'NA1_4905603838', 'NA1_4905768750', 'NA1_4905540470', 'NA1_4905601485', 'NA1_4905684014', 'NA1_4905700624', 'NA1_4905732378', 'NA1_4905559478', 'NA1_4905571935', 'NA1_4905610517', 'NA1_4905616147', 'NA1_4905725243', 'NA1_4905578715', 'NA1_4905628754', 'NA1_4905630062', 'NA1_4905751913', 'NA1_4905576941', 'NA1_4905686069', 'NA1_4905681670', 'NA1_4905728421', 'NA1_4905574734', 'NA1_4905590886', 'NA1_4905662843', 'NA1_4905753585', 'NA1_4905765804', 'NA1_4905595900', 'NA1_4905688464', 'NA1_4905578110', 'NA1_4905

In [7]:
import numpy as np
## add data to database
augments = pd.DataFrame(columns=['2-1 Games', '2-1 Avg Placement', '3-2 Games', '3-2 Avg Placement', '4-2 Games', '4-2 Avg Placement', 'Total Games', 'Total Avg Placement'])
augments = augments.astype({
    '2-1 Games':int,
    '2-1 Avg Placement':float,
    '3-2 Games':int,
    '3-2 Avg Placement':float,
    '4-2 Games':int,
    '4-2 Avg Placement':float,
    'Total Games':int,
    'Total Avg Placement':float,
    })

data_dragon = requests.get('https://ddragon.leagueoflegends.com/cdn/13.24.1/data/en_US/tft-augments.json').json()
aug_ids = list(data_dragon['data'].keys())
aug_names = [data_dragon['data'][i]['name'] for i in list(data_dragon['data'].keys())]

augments.insert(0, "Augment ID", aug_ids)
augments.insert(0, "Augment Name", aug_names)

augments['2-1 Games'] = 0
augments['3-2 Games'] = 0
augments['4-2 Games'] = 0
augments['Total Games'] = 0


count = 1
for match_id in recent_match_ids:

    if count % 100 == 0:
        time.sleep(120)
        print("sleeping 120s")

    api_url = 'https://americas.api.riotgames.com/tft/match/v1/matches/'+ match_id + '?api_key=' + api_key
    r = requests.get(api_url).json()
    for i in range(0, 8):
        # add to database
        ags = ['', '', '']
        augs = r['info']['participants'][i]['augments']
        for i in range(0, len(augs)):
            ags[i] = augs[i]
        placement = r['info']['participants'][i]['placement']


        if ags[0] != '':
            avg_place_21 = augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'].values[0]
            games_21 = augments.loc[augments['Augment ID'] == ags[0], '2-1 Games'].values[0]
            avg_total_place_21 = augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'].values[0]
            games_total_21 = augments.loc[augments['Augment ID'] == ags[0], 'Total Games'].values[0]


            if pd.isna(avg_place_21):
                augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'] = ((avg_place_21 * games_21) + placement) / (games_21 + 1)

            if pd.isna(avg_total_place_21):
                augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'] = ((avg_total_place_21 * games_total_21) + placement) / (games_total_21 + 1)

            augments.loc[augments['Augment ID'] == ags[0], '2-1 Games'] += 1
            augments.loc[augments['Augment ID'] == ags[0], 'Total Games'] += 1

        if ags[1] != '':
            avg_place_32 = augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'].values[0]
            games_32 = augments.loc[augments['Augment ID'] == ags[1], '3-2 Games'].values[0]
            avg_total_place_32 = augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'].values[0]
            games_total_32 = augments.loc[augments['Augment ID'] == ags[1], 'Total Games'].values[0]

            if pd.isna(avg_place_32):
                augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'] = ((avg_place_32 * games_32) + placement) / (games_32 + 1)
            if pd.isna(avg_total_place_32):
                augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'] = ((avg_total_place_32 * games_total_32) + placement) / (games_total_32 + 1)

            augments.loc[augments['Augment ID'] == ags[1], '3-2 Games'] += 1
            augments.loc[augments['Augment ID'] == ags[1], 'Total Games'] += 1



        if ags[2] != '':
            avg_place_42 = augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'].values[0]
            games_42 = augments.loc[augments['Augment ID'] == ags[2], '4-2 Games'].values[0]
            avg_total_place_42 = augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'].values[0]
            games_total_42 = augments.loc[augments['Augment ID'] == ags[2], 'Total Games'].values[0]

            if pd.isna(avg_place_42):
                augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'] = ((avg_place_42 * games_42) + placement) / (games_42 + 1)
            if pd.isna(avg_total_place_42):
                augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'] = ((avg_total_place_42 * games_total_42) + placement) / (games_total_42 + 1)

            augments.loc[augments['Augment ID'] == ags[2], '4-2 Games'] += 1
            augments.loc[augments['Augment ID'] == ags[2], 'Total Games'] += 1

    count += 1

sleeping 120s


In [11]:
augments.to_excel('augments.xlsx')
print(augments.head(5))

        Augment Name                   Augment ID  2-1 Games  \
0        Salvage Bin      TFT6_Augment_SalvageBin         13   
1  March of Progress   TFT6_Augment_SlowAndSteady         17   
2     Portable Forge   TFT6_Augment_PortableForge         22   
3     Binary Airdrop   TFT6_Augment_BinaryAirdrop          0   
4     Recombobulator  TFT6_Augment_Recombobulator          0   

   2-1 Avg Placement  3-2 Games  3-2 Avg Placement  4-2 Games  \
0           5.076923          0                NaN          0   
1           4.294118          0                NaN          0   
2           5.545455          0                NaN          0   
3                NaN         17           5.176471         27   
4                NaN          5           4.800000          2   

   4-2 Avg Placement  Total Games  Total Avg Placement  
0                NaN           13             5.076923  
1                NaN           17             4.294118  
2                NaN           22             5.54545

In [61]:
# updating database
import os
import sys
sys.path.append('..')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "tftAPP.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()
from stats.models import Augment



for row in augments.iterrows():
    # print(row[1])

    new_games_total = row[1]['Total Games']
    new_games_two_one = row[1]['2-1 Games']
    new_games_three_two = row[1]['3-2 Games']
    new_games_four_two = row[1]['4-2 Games']

    new_avg_total = row[1]['Total Avg Placement']
    new_avg_total = 0.0 if pd.isna(new_avg_total) else new_avg_total
    new_avg_two_one = row[1]['2-1 Avg Placement']
    new_avg_two_one = 0.0 if pd.isna(new_avg_two_one) else new_avg_two_one
    new_avg_three_two = row[1]['3-2 Avg Placement']
    new_avg_three_two = 0.0 if pd.isna(new_avg_three_two) else new_avg_three_two
    new_avg_four_two = row[1]['4-2 Avg Placement']
    new_avg_four_two = 0.0 if pd.isna(new_avg_four_two) else new_avg_four_two

    prevaug = Augment.objects.get(name=row[1]['Augment Name'])

    games_total = prevaug.games_total
    games_two_one = prevaug.games_two_one
    games_three_two = prevaug.games_three_two
    games_four_two = prevaug.games_four_two

    avg_total = float(prevaug.avg_total)
    avg_two_one = float(prevaug.avg_two_one)
    avg_three_two = float(prevaug.avg_three_two)
    avg_four_two = float(prevaug.avg_four_two)

    avg_total = 0.0 if new_games_total + games_total == 0 else ((avg_total * games_total) + (new_avg_total * new_games_total)) / (new_games_total + games_total)
    avg_two_one = 0.0 if new_games_two_one + games_two_one == 0 else ((avg_two_one * games_two_one) + (new_avg_two_one * new_games_two_one)) / (new_games_two_one + games_two_one)
    avg_three_two = 0.0 if new_games_three_two + games_three_two == 0 else ((avg_three_two * games_three_two) + (new_avg_three_two * new_games_three_two)) / (new_games_three_two + games_three_two)
    avg_four_two = 0.0 if new_games_four_two + games_four_two == 0 else ((avg_four_two * games_four_two) + (new_avg_four_two * new_games_four_two)) / (new_games_four_two + games_four_two)

    games_total += new_games_total
    games_two_one += new_games_two_one
    games_three_two += new_games_three_two
    games_four_two += new_games_four_two


    Augment.objects.filter(name=row[1]['Augment Name']).update(games_total=games_total,
                                                               games_two_one=games_two_one,
                                                               games_three_two=games_three_two,
                                                               games_four_two=games_four_two,
                                                               avg_total=avg_total,
                                                               avg_two_one=avg_two_one,
                                                               avg_three_two=avg_three_two,
                                                               avg_four_two=avg_four_two)


    # aug = Augment(name = row[1]['Augment Name'], 
    #               games_total = new_games_total, 
    #               games_two_one = new_games_two_one,
    #               games_three_two = new_games_three_two,
    #               games_four_two = new_games_four_two,
    #               avg_total = 0.0 if pd.isna(new_avg_total) else new_avg_total,
    #               avg_two_one = 0.0 if pd.isna(new_avg_two_one) else new_avg_two_one,
    #               avg_three_two = 0.0 if pd.isna(new_avg_three_two) else new_avg_three_two,
    #               avg_four_two = 0.0 if pd.isna(new_avg_four_two) else new_avg_four_two)
    # aug.save()